# Exercício 1: Ações e Commodities
###  Implementando Uma Estratégia Replicante 
<sub>Uirá Caiado. 11 de Maio, 2016<sub>

#### Resumo

_Neste projeto vou implementar uma estratégia de hedging dinâmico auto-financiável que replique o payoff de diferentes derivativos. Devido à natureza aleatória do ativo objeto dos contratos testados, este será modelado como um processo estocástico. Uma vez implementado o modelo, será realizada um série de simulações para estudar a efetividade da estratégia._

## 1. Introdução

Nesta sessão dou uma breve descrição do que é uma EDE e declaro o problema que será abordado.

### 1.1. Equações Diferenciais Estocásticas

Devido à incerteza inerente ao mercado financeiro, é necessário levar em conta uma flutuação aleatória nos modelos criados para descrever diferentes instrumentos. Para tanto, frequentemente assume-se o comportamento de um ativo como sendo um [processo estocático](https://en.wikipedia.org/wiki/Stochastic_process). 

Como bem explicado neste [post](http://math.stackexchange.com/questions/1470686/in-laymans-terms-what-is-a-stochastic-process), um processo estocástico é uma maneira de representar como alguma coisa se desenvolve ao longo do tempo. A cada unidade de tempo, o ativo pode assumir uma de muitas posições possíveis, e cada uma destas posições tem uma probabilidade associada. Ainda que não seja possível antecipar qual trajeto o ativo vai seguir, é possível inferir quais caminhos são possíveis baseado nesta dinâmica.

Processos estocásticos são de grande valia quando não se sabe exatamente as regras que regem esta evolução ou quando extrair tais regras com exatidão ou é muito complicado ou muito caro computacionalmente. Assim, frequentemente se utiliza alguma fonte de aleatoriadade para ajudar a descrever a dinâmica de um ativo. Aqui entram as EDEs. \cite{Wilmont} oferece a seguinte intuição sobre a forma geral de Equações Diferenciais Estocásticas (EDE):

$$\mathrm{d}\, Alguma\,Coisa = Deterministico\,\mathrm{d}t + Aleatório\,\mathrm{d}X $$


Onde $Alguma\,Coisa$ é o que se quer modelar (como preço de uma ação, quantidade de pessoas chegando em um hospital e etc), $Deterministico$ uma função que descreve como  $Alguma\,Coisa$ muda quando se retira a incerteza e $Aleatório$ outra função que descreve quão incerto $Alguma\,Coisa$ coisa é e $X$ é uma variável aleatória com distribuição Normal Padrão.

### 1.1. O Problema

Considerando um ativo-objeto cujo a dinâmica do preço satisfaz a seguinte EDE:

$$\frac{\mathrm{d} S_t}{S_t}=\mu\cdot \mathrm{d}t + \sigma\cdot \mathrm{d}W_{t}$$

Será vendido um derivativo com característica européia cujo payoff é descrito por uma função qualquer $V_T=V(T, S_T)$, onde $T$ é o vencimento do derivativo. $S_T$ é o preço do ativo-objeto em $T$ e é possível negociar qualquer quantidade dele em qualquer instante. Não há custo de transação (corretagem, emolumento, bid-ask spread, etc) e posições vendidas a descoberto no subjacente são permitidas, não havendo custos associados.

Pede-se que se implemente uma estratégia dinâmica de hedging auto-financiável que replique o payoff do derivativo vendido. A estratégia replicante será rebalanceada $N$ vezes (incluíndo a carteira inicial).

A simulação deve ser feita para os payoffs abaixo. $K$ é o Strike da opção.
- $V(T, S_T)=ln(S_T)$
- $V(T, S_T)=(ln(S_T))^2$
- $V(T, S_T)=(S_T-K)^2$
- $V(T, S_T)=\mathbf{1}_{S_T > K}$
- $V(T, S_T)=max(S_T-K, 0)$


## 2. Definições

Nesta sessão vou calcular a solução analítica das opções sugeridas na última sessão e discretizar o a dinâmica do ativo objeto 

### 2.1. Discretizando o Processo Estocástico

Para realizar a simulação, implementarei o processo estocástico do ativo-objeto, que segue um Movimento Browniano Geométrico. Por tanto, será necessário encontrar a solução para a EDE proposta, que resultará no [processo desejado](https://en.wikipedia.org/wiki/Geometric_Brownian_motion). Assim, seguindo as notas de aula, tenho que


\begin{equation}
    \begin{aligned}
    \frac{\mathrm{d} S_t}{S_t} &= \mu\cdot \mathrm{d}t + \sigma\cdot \mathrm{d}W_{t}
    \end{aligned}
\end{equation}


Que corresponde a forma derivada da integral estocástica


\begin{equation}
    \begin{aligned}
    \int_{t}^{T}{\frac{\mathrm{d} S_t}{S_t}} &= \int_{t}^{T}{\mu\cdot \mathrm{d}t} + \int_{t}^{T}{\sigma\cdot \mathrm{d}W_{t}} 
    \end{aligned}
\end{equation}



Considerando o [Teorema de Ito-Doeblin](https://en.wikipedia.org/wiki/Itô%27s_lemma) (Lemma de Ito) sendo da forma

\begin{equation}
    \begin{aligned}
    \mathrm{d} f(t, S_t) = \frac{\mathrm{\partial} f}{\partial t} \mathrm{d} t + \frac{\mathrm{\partial} f}{\partial S_t} \mathrm{d} S_t + \frac{1}{2} \frac{\mathrm{\partial^2} f}{\mathrm{\partial} S_{t}^{2}} (\mathrm{d} S_t)^2
    \end{aligned}
\end{equation}

Se definir uma função $f(S_t) = ln(S_t)$, posso aplicar o Lema de Ito sobre ela. Considerando que $(\mathrm{d} t)^2 = \mathrm{d} t \cdot \mathrm{d} X = 0$, tenho que $(\mathrm{d} S_t)^2 = \sigma^2  \cdot S_{t}^{2} \cdot \mathrm{d} t$. Também, como não há termos dependentes de $t$ em $f$, não precisarei do termo relacionado a $\mathrm{d} t$. Assim:

\begin{equation}
    \begin{aligned}
    \mathrm{d} f(t, S_t) &= \frac{\mathrm{\partial} f}{\partial S_t} \mathrm{d} S_t + \frac{1}{2} \frac{\mathrm{\partial^2} f}{\mathrm{\partial} S_{t}^{2}} (\mathrm{d} S_t)^2 \\
    &= \frac{1}{S_t} \cdot \mathrm{d} S_t - \frac{\sigma^2}{2} \cdot S_{t}^{2} \cdot \frac{1}{S_{t}^{2}} \cdot  \mathrm{d} t 
    \end{aligned}
\end{equation}

Integrando dos instantes $t$ até $T$, obtenho:

\begin{equation}
    \begin{aligned}
    \int_{t}^{T} \mathrm{d} f(t, S_t) &= \int_{t}^{T} \frac{1}{S_t} \cdot \mathrm{d} S_t - \int_{t}^{T} \frac{\sigma^2}{2} \cdot \mathrm{d} t \\
    \int_{t}^{T} \frac{\mathrm{d} S_t}{S_t} &= \int_{t}^{T} \mathrm{d} f(t, S_t) + \int_{t}^{T} \frac{\sigma^2}{2} \cdot \mathrm{d} t \\
    &= f(S_T) - f(S_t) + \frac{\sigma^2}{2} \cdot (T - t) \\
    &= ln(S_T) - ln(S_t) + \frac{\sigma^2}{2} \cdot (T - t) \\
    &= ln(S_T/S_t) + \frac{\sigma^2}{2} \cdot (T - t)
    \end{aligned}
\end{equation}

Logo, substituíndo este resultado na integral estocástica da dinâmica da ação, chego que:

\begin{equation}
    \begin{aligned}
    ln(S_T/S_t) + \frac{\sigma^2}{2} \cdot (T - t) &= \int_{t}^{T}{\mu\cdot \mathrm{d}t} + \int_{t}^{T}{\sigma\cdot \mathrm{d}W_{t}} \\
     ln(S_T/S_t) + \frac{\sigma^2}{2} \cdot (T - t) &= \mu\cdot (T - t) + \sigma \cdot (W_{T} - W_{t})
    \end{aligned}
\end{equation}

Por fim, lembrando que o $W_t$ é um processo de Winer e possui incrementos independentes, estacionários e distribuição $N(0, t)$ por definição, posso considerar que $W_t - W_s = W_{t-s}$. Assim:

\begin{equation}
    \begin{aligned}
     ln(S_T/S_t) + \frac{\sigma^2}{2} \cdot (T - t) &= \mu\cdot (T - t) + \sigma \cdot W_{T-t} \\
     ln(S_T/S_t) &= \left (\mu - \frac{\sigma^2}{2}\right ) \cdot (T - t) + \sigma \cdot W_{T-t} \\
     S_T &= S_t \cdot e^{ \left (\mu - \frac{\sigma^2}{2}\right ) \cdot (T - t) + \sigma \cdot W_{T-t}}
    \end{aligned}
\end{equation}


Onde $W_{T-t} =  \epsilon_t \cdot \sqrt{T-t} $, sendo $\epsilon$ uma variável aleatório com distribuição normal padrão. Esta última forma __discretizada__ de $S_T$ é a que utilizarei na simulação. Esta solução também poderia ter sido encontrada pelo [método de Euler](http://www.columbia.edu/~mh2078/MCS04/MCS_SDEs.pdf).

### 2.2. Solução Analítica para as Opções Dadas
...



## 3. Replicando o Portfólio

In [2]:
# importa bibliotecas necessarias
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns


## 4. Conclusão

## 5. Reflexões

_Style notebook and change matplotlib defaults_

In [3]:
#loading style sheet
from IPython.core.display import HTML
HTML( open('ipython_style.css').read())

In [1]:
#changing matplotlib defaults
%matplotlib inline
import seaborn as sns
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("Set1", 10))

In [4]:
HTML(open('main.js').read())